In [707]:
# CHECK CATEGOIRES FUNCTION
def checkCategorical(data_frame, column, approved_list):
    _counts = df[column].value_counts()
    not_in_list = 0
    for x in _counts.index.tolist():
        if x not in approved_list: 
            print(x)
            not_in_list += 1
    print('{} {} categories found that were not in the official {} list.'.format(not_in_list, column, column))
    print(_counts)

In [708]:
# PLOT CONTINUOUS DATA FUNCTION
def plotContinuous(data_frame, x, y, marker_size, size):
    groups = data_frame.groupby('y')
    p = 0
    for name, group in groups:
        plt.plot(group[x],group[y],marker='.',ms=marker_size[p],linestyle='',label=name)
        p += 1
    plt.rcParams["figure.figsize"] = (size[0],size[1])
    plt.xlabel=x
    plt.ylabel=y
    plt.legend(prop={'size':20})
    plt.show()

In [709]:
# MLP WRAPPER
def mlpWrapper(df):
    # split into train and test data
    print('TTT seed value: {}'.format(TTT_seed))
    train, test = train_test_split(
        df,
        test_size=0.2,
        random_state=TTT_seed
    )
    X_train = train[train.columns[:-1]]
    X_test = test[test.columns[:-1]]
    y_train = train[train.columns[-1]]
    y_test = test[test.columns[-1]]

    # useful info print out
    print('training set size: {}'.format(train.shape[0]))
    print('testing set size: {}'.format(test.shape[0]))

    # call training function
    print('MLP seed value: {}'.format(MLP_seed))
    mlp = MLPClassifier(
        hidden_layer_sizes=50,
        activation='logistic',
        learning_rate_init=0.1,
        batch_size=100,
        solver='sgd',
        random_state=MLP_seed,
        max_iter=10000
    ).fit(X_train,y_train)

    # print out results
    result = mlp.predict(test[test.columns[:-1]])
    print('Training accuracy: {:.2f}'.format(mlp.score(X_train, y_train)))
    print('Testing accuracy: {:.2f}'.format(mlp.score(X_test, y_test)))

In [710]:
# 1 HOT
def oneHot(df, column):

    # 1 hot encoding
    _df = pd.get_dummies(df, columns=[column], prefix=column)
    
    #return
    return _df

In [711]:
# INSERT
def insert(df, insert_df):
    _left = df[df.columns[:-1]]
    _right = df[df.columns[-1]]
    _final = pd.concat([_left,insert_df,_right],axis=1)
    return _final

In [712]:
# NORMALIZED DATA
def normalizedDf(df):

    # selection list for readability
    selection = [
        'age',
        'campaign',
        'pdays',
        'previous',
        'emp.var.rate',
        'cons.price.idx',
        'cons.conf.idx',
        'euribor3m',
        'nr.employed',
        'y'
    ]

    # scrub dataframe: no first selection and no first time subs
    _df = df[selection]

    # max/min normalization
    _norm = _df[_df.columns[:-1]]
    _norm = (_norm - _norm.mean()) / (_norm.max() - _norm.min())
    
    # reassemble
    _final = pd.concat([_norm, _df['y']], axis=1)
    
    #return
    return _final

In [713]:
# REMOVE FIRST TIME CALLS
def removeFirstCalls(df):
    _df = df
    return _df[_df['pdays']!=999]

In [714]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from random import randint

In [715]:
# LOAD DATA
file_name = '../bank-additional/bank-additional/bank-additional-full.csv'
df = pd.read_csv(file_name, sep=';')
df = df.drop_duplicates(keep='first')
TTT_seed = randint(0,99)
MLP_seed = randint(0,99)
_ready = {}

In [716]:
# MLP NORMALIZED DATAFRAME 

# remove first call subs
_normalized = removeFirstCalls(df)

# normalize
_normalized = normalizedDf(_normalized)

# add to ready
_ready['normalized'] = _normalized

# output
print(_normalized.columns)

Index(['age', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


In [717]:
# # MLP ONLY FIRST CALL SUBS

# # selection list for readability
# selection = [
#     'age',
#     'duration',
#     'campaign',
#     'pdays',
#     'previous',
#     'emp.var.rate',
#     'cons.price.idx',
#     'cons.conf.idx',
#     'euribor3m',
#     'nr.employed',
#     'y'
# ]

# # scrub dataframe: no first selection and only first call subs
# scrubbed = df[selection]
# scrubbed = scrubbed[scrubbed['pdays']==999]

# # max/min normalization
# norm_df = scrubbed[selection[:-1]]
# norm_df = norm_df.loc[:, norm_df.columns != 'pdays']
# norm_df = (norm_df-norm_df.mean())/(norm_df.max()-norm_df.min())
# final = pd.concat([norm_df,scrubbed['pdays'],scrubbed['y']],axis=1)

# # split into train and test data
# train,test = train_test_split(final,test_size=0.999, random_state=24)
# X_train = train[train.columns[:-1]]
# X_test = test[test.columns[:-1]]
# y_train = train[train.columns[-1]]
# y_test = test[test.columns[-1]]

# # useful info print out
# print('training set size: {}'.format(train.shape[0]))
# print('testing set size: {}'.format(test.shape[0]))

# # call training function
# mlp = MLPClassifier(
#     hidden_layer_sizes=1,
#     activation='logistic',
#     learning_rate_init=0.04,
#     batch_size=10,
#     solver='sgd',
#     random_state=0,
#     max_iter=10000
# ).fit(X_train,y_train)

# # print out results
# result = mlp.predict(test[test.columns[:-1]])
# print('Training accuracy: {:.2f}'.format(mlp.score(X_train, y_train)))
# print('Testing accuracy: {:.2f}'.format(mlp.score(X_test, y_test)))

In [718]:
# THE PROBLEM WITH FIRST TIME CALL DATA
counts = df.pdays.value_counts()
print('First time calls represented {:.0%} of the total number of calls.'.format(counts.max()/df.shape[0]))
no = df[df['y']=='no']
counts = no.pdays.value_counts()
print('{:.0%} of the calls were first time calls where the customer declined the offer.'.format(counts.max()/df.shape[0]))

First time calls represented 96% of the total number of calls.
87% of the calls were first time calls where the customer declined the offer.


In [719]:
# MLP ENCODED JOB

# scrub first time calls
_jobs = removeFirstCalls(df)

# 1 hot encoding
_jobs = _jobs['job']
print(_jobs.value_counts())
_jobs = oneHot(_jobs, 'job')

# add to ready
_ready['job'] = _jobs

# output
# print(_jobs.head())

admin.           469
technician       235
retired          175
student          140
blue-collar      129
management       107
services          76
unemployed        65
housemaid         41
self-employed     32
entrepreneur      26
unknown           20
Name: job, dtype: int64


In [720]:
# MLP ENCODED MARITAL

# scrub first time calls
_marital = removeFirstCalls(df)

# 1 hot encoding
_marital = _marital['marital']
print(_marital.value_counts())
_marital = oneHot(_marital, 'marital')
_marital = _marital.drop('marital_unknown',axis=1)

# add to ready
_ready['marital'] = _marital

# output
# print(_marital.head())

married     797
single      572
divorced    141
unknown       5
Name: marital, dtype: int64


In [721]:
# MLP ENCODED EDUCATION

# scrub first time calls
_education = removeFirstCalls(df)

# replace
_encoded_education = {
    'education': {
        'unknown': 0/7,
        'illiterate': 1/7,
        'basic.4y': 2/7,
        'basic.6y': 3/7,
        'basic.9y': 4/7,
        'high.school': 5/7,
        'professional.course': 6/7,
        'university.degree': 7/7,
    }
}
print(_education.education.value_counts())
_education = _education.replace(_encoded_education)
_education = _education.education

# add to ready
_ready['education'] = _education

# output
# print(_education.head())

university.degree      578
high.school            332
professional.course    202
basic.4y               140
basic.9y               121
unknown                 98
basic.6y                43
illiterate               1
Name: education, dtype: int64


In [722]:
# MLP ENCODED DEFAULT

# scrub first time calls
_default = removeFirstCalls(df)

# 1 hot encoding
_default = _default.default
print(_default.value_counts())
_default = oneHot(_default, 'default')
_default = _default.drop('default_unknown',axis=1)

# add to ready
_ready['default'] = _default

# output
# print(_default.head())

no         1451
unknown      64
Name: default, dtype: int64


In [723]:
# MLP ENCODED HOUSING

# scrub first time calls
_housing = removeFirstCalls(df)

# 1 hot encoding
_housing = _housing.housing
print(_housing.value_counts())
_housing = oneHot(_housing, 'housing')
_housing = _housing.drop('housing_unknown',axis=1)

# add to ready
_ready['housing'] = _housing

# output
# print(_housing.head())

yes        835
no         646
unknown     34
Name: housing, dtype: int64


In [724]:
# MLP ENCODED LOAN

# scrub first time calls
_loan = removeFirstCalls(df)

# 1 hot encoding
_loan = _loan.loan
print(_loan.value_counts())
_loan = oneHot(_loan, 'loan')
_loan = _loan.drop('loan_unknown',axis=1)

# add to ready 
_ready['loan'] = _loan

# output
# print(_loan.head())

no         1251
yes         230
unknown      34
Name: loan, dtype: int64


In [725]:
# MLP ENCODED CONTACT

# scrub first time calls
_contact = removeFirstCalls(df)

# 1 hot encoding
_contact = _contact.contact
print(_contact.value_counts())
_contact = oneHot(_contact, 'contact')

# add to ready
_ready['contact'] = _contact

# output
# print(_contact.head())

cellular     1402
telephone     113
Name: contact, dtype: int64


In [726]:
# MLP ENCODED MONTH

# scrub first time calls
_month = removeFirstCalls(df)

# 1 hot encoding
_month = _month.month
print(_month.value_counts())
_month = oneHot(_month, 'month')

# add to ready
_ready['month'] = _month

# output
# print(_month.head())

may    248
aug    236
nov    190
sep    161
oct    157
jun    154
jul    118
apr    115
mar     90
dec     46
Name: month, dtype: int64


In [727]:
# MLP ENCODED DAY OF WEEK

# scrub first time calls
_day_of_week = removeFirstCalls(df)

# 1 hot encoding
_day_of_week = _day_of_week.day_of_week
print(_day_of_week.value_counts())
_day_of_week = oneHot(_day_of_week, 'day_of_week')

# add to ready
_ready['day_of_week'] = _day_of_week

# output
# print(_day_of_week.head())

thu    343
tue    320
mon    310
wed    296
fri    246
Name: day_of_week, dtype: int64


In [728]:
# MLP ENCODED PREVIOUS OUTCOME

# scrub first time calls
_poutcome = removeFirstCalls(df)

# 1 hot encoding
_poutcome = _poutcome.poutcome
print(_poutcome.value_counts())
_poutcome = oneHot(_poutcome, 'poutcome')

# add to ready
_ready['poutcome'] = _poutcome

# output
# print(_poutcome.head())

success    1373
failure     142
Name: poutcome, dtype: int64


In [729]:
# COMBINE AND TEST

# just numeric
print('NORMALIZED NUMERICAL DATA ONLY (NO FIRST CALLS):')
mlpWrapper(_normalized)
print()

NORMALIZED NUMERICAL DATA ONLY (NO FIRST CALLS):
TTT seed value: 73
training set size: 1212
testing set size: 303
MLP seed value: 86
Training accuracy: 0.68
Testing accuracy: 0.68

adding job...
adding marital...
adding education...
adding default...
adding housing...
adding loan...
adding contact...
adding month...
adding day_of_week...
adding poutcome...

ALL DATA ONLY (NO FIRST CALLS):
TTT seed value: 73
training set size: 1212
testing set size: 303
MLP seed value: 86
Training accuracy: 0.70
Testing accuracy: 0.66


In [ ]:
# add encoded data
_all_data = _normalized
for _key in _ready:
    if _key != 'normalized':
        print('adding {}...'.format(_key))
        _all_data = insert(_all_data, _ready[_key])
print()

# all data
print('ALL DATA ONLY (NO FIRST CALLS):')
mlpWrapper(_all_data)